In [1]:
!pip install catboost
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 2.2 MB/s eta 0:00:0000:0100:02


In [2]:
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

2024-05-17 13:42:07.473702: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 13:42:07.543972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 13:42:07.544018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 13:42:07.546522: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 13:42:07.559501: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 13:42:07.560882: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
# Đọc dữ liệu từ file data.pkl.File nay chua du lieu da duoc can bang voi WGAN va ENN-Kmean
with open('cse-cic-ids2018-balanced.pkl', 'rb') as file:
    data_loaded = pickle.load(file)

X_train = data_loaded['X_train']
X_test = data_loaded['X_test']
y_train = data_loaded['y_train']
y_test = data_loaded['y_test']


In [4]:
unique_values, counts = np.unique(y_train, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

0: 7000
1: 7000
2: 7000
3: 7000
4: 7000


In [5]:
unique_values, counts = np.unique(y_test, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

0.0: 183
1.0: 69
2.0: 519
3.0: 3000
4.0: 3000


In [6]:
X_train[:10]

array([[1.22074038e-03, 3.52941176e-01, 9.58525925e-02, 3.32594235e-04,
        2.08539701e-04, 6.75264732e-05, 2.37231481e-05, 9.15580385e-03,
        0.00000000e+00, 8.92354041e-03, 1.60312297e-02, 2.44460857e-01,
        0.00000000e+00, 1.13516227e-01, 2.48991856e-01, 8.38257082e-08,
        1.69711126e-07, 1.36941931e-02, 3.36699180e-02, 5.41503253e-02,
        3.33356879e-08, 5.41590289e-02, 1.80542697e-02, 4.44513841e-02,
        5.41529833e-02, 2.23349109e-06, 9.58527168e-02, 3.19740151e-02,
        4.02641956e-02, 5.41857564e-02, 9.84880040e-06, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.63673629e-04,
        3.54506402e-04, 8.40103992e-08, 1.73878086e-07, 0.00000000e+00,
        1.02731223e-02, 4.18128565e-02, 2.61466251e-02, 6.83646002e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.93650794e-03, 4.70210167e-02, 8.92354041e-03, 1.135162

In [7]:
y_train[:5]

array([3, 3, 3, 3, 3])

In [8]:
# Assuming X_train, X_test, y_train, y_test are already defined and preprocessed.
num_classes = len(np.unique(np.concatenate([y_train, y_test])))

# Define and train models
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)

cbt = CatBoostClassifier(verbose=0)
cbt.fit(X_train, y_train)

gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)

bme = BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=10)
bme.fit(X_train, y_train)

# Deep Neural Network
dnn = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
dnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
dnn.fit(X_train, to_categorical(y_train, num_classes=num_classes), epochs=10, batch_size=32, verbose=0)

# Predict probabilities
prob_xgb = xgb.predict_proba(X_test)
prob_cbt = cbt.predict_proba(X_test)
prob_gbm = gbm.predict_proba(X_test)
prob_bme = bme.predict_proba(X_test)
prob_dnn = dnn.predict(X_test)  # This already gives probabilities due to softmax



/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


212/212 [==============================] - 0s 1ms/step


In [9]:
def evaluate_model(y_true, predictions):
    acc = accuracy_score(y_true, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, predictions, average='weighted')
    return acc, precision, recall, f1

In [10]:
# Convert predicted probabilities to class labels for evaluation
predictions_xgb = np.argmax(prob_xgb, axis=1)
predictions_cbt = np.argmax(prob_cbt, axis=1)
predictions_gbm = np.argmax(prob_gbm, axis=1)
predictions_bme = np.argmax(prob_bme, axis=1)
predictions_dnn = np.argmax(prob_dnn, axis=1)

# Evaluate each model
results_xgb = evaluate_model(y_test, predictions_xgb)
results_cbt = evaluate_model(y_test, predictions_cbt)
results_gbm = evaluate_model(y_test, predictions_gbm)
results_bme = evaluate_model(y_test, predictions_bme)
results_dnn = evaluate_model(y_test, predictions_dnn)

# Print results for each model
print("XGB - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_xgb))
print("CBT - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_cbt))
print("GBM - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_gbm))
print("BME - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_bme))
print("DNN - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_dnn))


XGB - Accuracy: 0.9981, Precision: 0.9981, Recall: 0.9981, F1-Score: 0.9981
CBT - Accuracy: 0.9987, Precision: 0.9987, Recall: 0.9987, F1-Score: 0.9987
GBM - Accuracy: 0.9990, Precision: 0.9990, Recall: 0.9990, F1-Score: 0.9990
BME - Accuracy: 0.9987, Precision: 0.9987, Recall: 0.9987, F1-Score: 0.9987
DNN - Accuracy: 0.9935, Precision: 0.9940, Recall: 0.9935, F1-Score: 0.9936


In [11]:
# Ensemble learning

# Weighted average of probabilities
ensemble_probabilities = np.average(
    [prob_xgb, prob_cbt, prob_gbm, prob_bme, prob_dnn],
    axis=0,
    weights=[0.3, 0.2, 0.2, 0.2, 0.1]
)

# Final prediction is the class with the highest average probability
ensemble_predictions = np.argmax(ensemble_probabilities, axis=1)

# Evaluate ensemble
ensemble_results = evaluate_model(y_test, ensemble_predictions)

print("Ensemble Accuracy:", ensemble_results[0])
print("Ensemble Precision:", ensemble_results[1])
print("Ensemble Recall:", ensemble_results[2])
print("Ensemble F1-Score:", ensemble_results[3])

Ensemble Accuracy: 0.9989661792940482
Ensemble Precision: 0.9989779465286168
Ensemble Recall: 0.9989661792940482
Ensemble F1-Score: 0.9989682491275246
